In [1]:
var('eps', latex_name = '\\varepsilon')
var('t')
var('r')
var('th', latex_name = '\\theta')
var('ph', latex_name = '\\varphi')
coords = (t,r,th,ph)

h = function('h')(r,th)
k = function('k')(r,th)
m = function('m')(r,th)

omega = function('omega', latex_name='\\omega')(r)

nu = function('nu', latex_name='\\nu')(r)
lamb = function('lamb', latex_name='\\lambda')(r)

# <center> Tensor de Einstein </center>

In [2]:
gdd = matrix(SR, 4, 4)

gdd[0,0] = -exp(nu)
gdd[1,1] = exp(lamb)
gdd[2,2] = r^2
gdd[3,3] = r^2*sin(th)^2

guu = 1/gdd

K1 = matrix(SR, 4, 4)
K1[0,3] = -omega*r^2*sin(th)^2
K1[3,0] = -omega*r^2*sin(th)^2

K2 = matrix(SR, 4, 4)
K2[0,0] = -4*exp(nu)*h + 2*r^2*sin(th)^2*omega^2
K2[1,1] = 4*exp(lamb)*m
K2[2,2] = 4*k*r^2
K2[3,3] = 4*k*r^2*sin(th)^2

In [3]:
gedd = matrix(SR, 4, 4)
for ii in range(0,4):
    for jj in range(0,4):
        gedd[ii,jj] = gdd[ii,jj] + eps*K1[ii,jj] + 1/2*eps^2*K2[ii,jj]
        
geuu = 1/gedd
    
chris = [[[0 for kk in range(0,4)] for jj in range(0,4)] for ii in range(0,4)]
for ii in range(0,4):
    for jj in range(0,4):
        for kk in range(0,4):
            chris[ii][jj][kk] = 0.5*sum(geuu[ii,dd]*(diff(gedd[dd,jj],coords[kk])
                                                   + diff(gedd[dd,kk],coords[jj])
                                                   - diff(gedd[jj,kk],coords[dd])) for dd in range(0,4))

riem = [[[[0 for ll in range(0,4)] for kk in range(0,4)] for jj in range(0,4)] for ii in range(0,4)]
for ii in range(0,4):
    for jj in range(0,4):
        for kk in range(0,4):
            for ll in range(0,4):
                riem[ii][jj][kk][ll] = (diff(chris[ii][jj][ll],coords[kk])
                                      - diff(chris[ii][jj][kk],coords[ll])
                                      + sum(chris[ii][kk][dd]*chris[dd][ll][jj]
                                      - chris[ii][ll][dd]*chris[dd][kk][jj] for dd in range(0,4)))

ricci = matrix(SR, 4, 4)
for ii in range(0,4):
    for kk in range(0,4):
        ricci[ii,kk] = sum(riem[dd][ii][dd][kk] for dd in range(0,4))

s_curv = sum(sum(ricci[ii,jj]*geuu[ii,jj] for ii in range(0,4)) for jj in range(0,4))

Ge = matrix(SR, 4, 4)
Ge = ricci - 0.5*gedd*s_curv

dGe = matrix(SR, 4, 4)
ddGe = matrix(SR, 4, 4)
for ii in range(0,4):
    for jj in range(0,4):
        dGe[ii,jj] = diff(Ge[ii,jj],eps)
        ddGe[ii,jj] = diff(Ge[ii,jj],eps,eps)

# <center> Tensor de energía-momento </center>

In [4]:
Omegae = function('Omegae', latex_name='\\Omega_{\\varepsilon}')(eps)
Ne = function('Ne', latex_name='N_e')(r,th,eps)

ue = [0,0,0,0]
u = [0,0,0,0]

ue[0] = Ne;         ue[1] = 0;   ue[2] = 0;   ue[3] = Ne*Omegae
u[0] = exp(-nu/2);  u[1] =  0;   u[2] = 0;    u[3] =  0

Ne_value = solve(u[0]==ue[0], Ne)[0].rhs()
Omegae_value = solve(u[3]==ue[3], Omegae)[0].rhs()

u1 = [0,0,0,0]
u2 = [0,0,0,0]

u10 = function('u10',latex_name="u^{(1)0}")(r,th)
u20 = function('u20',latex_name="u^{(2)0}")(r,th)

var('Omega1', latex_name="\\Omega^{(1)}")

sus_func = [eps == 0,
            Ne.subs(eps==0) == Ne_value,
            Omegae.subs(eps==0) == Omegae_value,
            diff(Ne,eps,eps).subs(eps==0) == u20,
            diff(Omegae,eps,eps).subs(eps==0) == 0, #En nuestro gauge Omega2=0
            diff(Ne,eps).subs(eps==0) == u10,
            diff(Omegae,eps).subs(eps==0) == Omega1]

for ii in range(0,4):
    u1[ii] = diff(ue[ii],eps)
    u2[ii] = diff(ue[ii],eps,eps)

Ue = [0,0,0,0]
for ii in range(0,4):
    for jj in range(0,4):
        Ue[ii] += ue[jj]*gedd[jj,ii]

geUeue = 0
for ii in range(0,4):
    geUeue += Ue[ii]*ue[ii]

ddgeUeue = diff(geUeue, eps, eps).subs(sus_func) == 0
u20_value = solve(ddgeUeue, u20)[0].rhs()

dgeUeue = diff(geUeue, eps).subs(sus_func) == 0
u10_value = solve(dgeUeue, u10)[0].rhs()

sus_func.extend([u20 == u20_value,
                 u10 == u10_value])

for ii in range(0,4):
    for jj in range(0,len(sus_func)):
        u1[ii] = u1[ii].subs(sus_func[jj])
        u2[ii] = u2[ii].subs(sus_func[jj])

u1_paper = [0,0,0,0]
u1_paper[0] = 1/2*e^(-3*nu/2)*K1[0,0]
u1_paper[3] = e^(-nu/2)*Omega1

u2_paper = [0,0,0,0]
u2_paper[0] = e^(-3*nu/2)*(1/2*K2[0,0]+3/4*e^(-nu)*K1[0,0]+2*Omega1*K1[0,3]+Omega1^2*r^2*sin(th)^2)
u2_paper[3] = e^(-nu/2)*(e^(-nu)*K1[0,0]*Omega1)  

u1_check = [ii-jj for (ii,jj) in zip(u1, u1_paper)]
u2_check = [ii-jj for (ii,jj) in zip(u2, u2_paper)]

show(LatexExpr("u^{(1)} = "), u1, LatexExpr("\\rightarrow"), u1_check==[0,0,0,0])
show(LatexExpr("u^{(2)} = "), u2, LatexExpr("\\rightarrow"), u2_check==[0,0,0,0])

u^{(1)} =  [0, 0, 0, Omega1*e^(-1/2*nu(r))] \rightarrow True

u^{(2)} =  [((Omega1^2*r^2 - 2*Omega1*r^2*omega(r) + r^2*omega(r)^2)*sin(th)^2 - 2*e^nu(r)*h(r, th))*e^(-3/2*nu(r)),
 0,
 0,
 0] \rightarrow True

In [5]:
Ee = function('Ee', latex_name='E_{\\varepsilon}')(r,th,eps)
Pe = function('Pe', latex_name='P_{\\varepsilon}')(r,th,eps)

Te = matrix(SR, 4, 4)
dTe = matrix(SR, 4, 4)
ddTe = matrix(SR, 4, 4)
for ii in range(0,4):
    for jj in range(0,4):
        Te[ii,jj] = (Ee+Pe)*Ue[ii]*Ue[jj]+Pe*gedd[ii,jj]
        dTe[ii,jj] = diff(Te[ii,jj],eps)
        ddTe[ii,jj] = diff(Te[ii,jj],eps,eps)

In [6]:
kappa = 8*pi
E = function('E')(r,th)
P = function('P')(r,th)

sus_func.extend([Ee.subs(eps==0) == E,
                 Pe.subs(eps==0) == P])

G = copy(Ge)
T = copy(Te)
for ii in range(0,len(sus_func)):
    G = G.subs(sus_func[ii])
    T = T.subs(sus_func[ii])

In [7]:
show(LatexExpr("T_{\\varepsilon}\lvert_0 ="), T)

T_{\varepsilon}\lvert_0 = [(E(r, th) + P(r, th))*e^nu(r) - P(r, th)*e^nu(r)                                                0                                                0                                                0]
[                                               0                               P(r, th)*e^lamb(r)                                                0                                                0]
[                                               0                                                0                                     r^2*P(r, th)                                                0]
[                                               0                                                0                                                0                           r^2*P(r, th)*sin(th)^2]

# <center> $G_{\mu\nu} = \kappa T_{\mu\nu}$ </center>

In [8]:
E_value_00 = solve(G[0,0] == kappa*T[0,0], E)[0].rhs()
P_value_11 = solve(G[1,1] == kappa*T[1,1], P)[0].rhs()
P_value_22 = solve(G[2,2] == kappa*T[2,2], P)[0].rhs()
P_value_33 = solve(G[3,3] == kappa*T[3,3], P)[0].rhs()
ddnu_value = solve(P_value_11 == P_value_22, diff(nu,r,r))[0].rhs()

sus_func.extend([diff(nu,r,r) == ddnu_value])

dE_value_00 = diff(E_value_00,r)
dP_value_11 = diff(P_value_11,r)
for ii in range(0,len(sus_func)):
    dE_value_00 = dE_value_00.subs(sus_func[ii])
    dP_value_11 = dP_value_11.subs(sus_func[ii])

sus_func.extend([E == E_value_00,
                 diff(E,r) == dE_value_00,
                 P == P_value_11,
                 diff(P,r) == dP_value_11])

E_paper = diff(lamb,r) - 1/r*(1-e^(lamb)) - r*e^(lamb)*8*pi*E == 0
E_paper = solve(E_paper, E)[0].rhs()
E_check = E_value_00 - E_paper

P_paper = diff(nu,r) - 1/r*(-1+e^(lamb)) - r*e^(lamb)*8*pi*P == 0
P_paper = solve(P_paper, P)[0].rhs()
P_check = P_value_11 - P_paper

ddnu_paper = 2*r*diff(nu,r,r) + diff(nu,r)*(r*diff(nu,r)-2)-diff(lamb,r)*(2+r*diff(nu,r))+4/r*(e^(lamb)-1) == 0
ddnu_paper = solve(ddnu_paper, diff(nu,r,r))[0].rhs()
ddnu_check = ddnu_value - ddnu_paper

show(LatexExpr("E = "), E_value_00, LatexExpr("\\rightarrow"), bool(E_check==0))
show(LatexExpr("P = "), P_value_11, LatexExpr("\\rightarrow"), bool(P_check==0))
show(LatexExpr("\\nu'' ="), ddnu_value, LatexExpr("\\rightarrow"), bool(ddnu_check==0))

E =  1/8*(r*e^(-lamb(r))*diff(lamb(r), r) - e^(-lamb(r)) + 1)/(pi*r^2) \rightarrow True

P =  1/8*(r*e^(-lamb(r))*diff(nu(r), r) + e^(-lamb(r)) - 1)/(pi*r^2) \rightarrow True

\nu'' = -1/2*(r^2*diff(nu(r), r)^2 - 2*r*diff(lamb(r), r) - (r^2*diff(lamb(r), r) + 2*r)*diff(nu(r), r) + 4*e^lamb(r) - 4)/r^2 \rightarrow True

# <center> $G^{\prime}_{\mu\nu} = \kappa T^{\prime}_{\mu\nu}$ </center>

In [9]:
j = e^(-(lamb+nu)/2)
M = r*(1-e^(-lamb))/2

E1 = function('E1', latex_name='E^{(1)}')(r,th)
P1 = function('P1', latex_name='P^{(1)}')(r,th)

sus_func.extend([diff(Ee,eps).subs(eps==0) == E1,
                 diff(Pe,eps).subs(eps==0) == P1])

dG = copy(dGe)
dT = copy(dTe)
for ii in range(0,len(sus_func)):
    dG = dG.subs(sus_func[ii])
    dT = dT.subs(sus_func[ii])

In [10]:
dEq_00 = dG[0,0] - kappa*dT[0,0] == 0

for ii in range(0,len(sus_func)):
    dEq_00 = dEq_00.subs(sus_func[ii])

E1_value = solve(dEq_00,E1)[0].rhs().simplify_full()
sus_func.extend([E1==E1_value])

E1_paper = 0
E1_check = E1_value - E1_paper

show(LatexExpr("E^{(1)} = "), E1_value, LatexExpr("\\rightarrow"), bool(E1_check==0))

E^{(1)} =  0 \rightarrow True

In [11]:
dEq_11 = dG[1,1] - kappa*dT[1,1] == 0
dEq_22 = dG[2,2] - kappa*dT[2,2] == 0
dEq_33 = dG[3,3] - kappa*dT[3,3] == 0

P1_value = solve(dEq_11,P1)[0].rhs().simplify_full()
sus_func.extend([P1==P1_value])

P1_paper = 0
P1_check = P1_value - P1_paper

show(LatexExpr("P^{(1)} = "), P1_value, LatexExpr("\\rightarrow"), bool(P1_check==0))

P^{(1)} =  0 \rightarrow True

In [12]:
dEq_03 = dG[0,3] - kappa*dT[0,3] == 0

for ii in range(0,len(sus_func)):
    dEq_03 = dEq_03.subs(sus_func[ii])

ddomega_value = solve(dEq_03, diff(omega,r,r))[0].rhs().simplify_full()
sus_func.extend([diff(omega,r,r) == ddomega_value])

dEq_03_paper = diff(r^4*j*diff(omega,r),r)\
              + (r^2*j*e^(lamb)/sin(th)^3)*diff(sin(th)^3*diff(omega,th),th)\
              + 4*r^3*diff(j,r)*(omega-Omega1) == 0

ddomega_paper = solve(dEq_03_paper, diff(omega,r,r))[0].rhs()
ddomega_check = ddomega_value - ddomega_paper
show(LatexExpr("\\omega'' = "), ddomega_value, LatexExpr("\\rightarrow"), bool(ddomega_check==0))

\omega'' =  -1/2*(4*(Omega1 - omega(r))*diff(lamb(r), r) + 4*(Omega1 - omega(r))*diff(nu(r), r) - (r*diff(lamb(r), r) + r*diff(nu(r), r) - 8)*diff(omega(r), r))/r \rightarrow True

# <center> $G^{\prime\prime}_{\mu\nu} = \kappa T^{\prime\prime}_{\mu\nu}$ </center>

In [13]:
E2 = function('E2', latex_name='E^{(2)}')(r,th)
P2 = function('P2', latex_name='P^{(2)}')(r,th)

sus_func.extend([diff(Ee,eps,eps).subs(eps==0) == E2,
                 diff(Pe,eps,eps).subs(eps==0) == P2])

ddG = copy(ddGe)
ddT = copy(ddTe)
for ii in range(0,len(sus_func)):
    ddG = ddG.subs(sus_func[ii])
    ddT = ddT.subs(sus_func[ii])

## <center> Descomposición en $P_l(\cos(\theta))$ </center>

In [14]:
h0 = function('h0', latex_name='h_0')(r)
m0 = function('m0', latex_name='m_0')(r)

h2 = function('h2', latex_name='h_2')(r)
k2 = function('k2', latex_name='k_2')(r)
m2 = function('m2', latex_name='m_2')(r)

E20 = function('E20', latex_name='E_{0}^{(2)}')(r)
P20 = function('P20', latex_name='P_{0}^{(2)}')(r)

E22 = function('E22', latex_name='E_{2}^{(2)}')(r)
P22 = function('P22', latex_name='P_{2}^{(2)}')(r)

LP1 = function('LP1', latex_name='LP_1')(th)
LP2 = function('LP2', latex_name='LP_2')(th)

lp2 = (3*cos(th)^2-1)/2

In [15]:
sus_descomp = [m==m0+m2*lp2,
               diff(m,r)==diff(m0+m2*lp2,r),
               diff(m,r,r)==diff(m0+m2*lp2,r,r),
               diff(m,th)==diff(m0+m2*lp2,th),
               diff(m,th,th)==diff(m0+m2*lp2,th,th),
               diff(m,th,r)==diff(m0+m2*lp2,th,r),
               diff(m,r,th)==diff(m0+m2*lp2,r,th),
               k==k2*lp2,
               diff(k,r)==diff(k2*lp2,r),
               diff(k,r,r)==diff(k2*lp2,r,r),
               diff(k,th)==diff(k2*lp2,th),
               diff(k,th,th)==diff(k2*lp2,th,th),
               diff(k,th,r)==diff(k2*lp2,th,r),
               diff(k,r,th)==diff(k2*lp2,r,th),
               h==h0+h2*lp2,
               diff(h,r)==diff(h0+h2*lp2,r),
               diff(h,r,r)==diff(h0+h2*lp2,r,r),
               diff(h,th)==diff(h0+h2*lp2,th),
               diff(h,th,th)==diff(h0+h2*lp2,th,th),
               diff(h,th,r)==diff(h0+h2*lp2,th,r),
               diff(h,r,th)==diff(h0+h2*lp2,r,th),
               E2==E20+E22*lp2,
               diff(E2,r)==diff(E20+E22*lp2,r),
               diff(E2,r,r)==diff(E20+E22*lp2,r,r),
               diff(E2,th)==diff(E20+E22*lp2,th),
               diff(E2,th,th)==diff(E20+E22*lp2,th,th),
               diff(E2,th,r)==diff(E20+E22*lp2,th,r),
               diff(E2,r,th)==diff(E20+E22*lp2,r,th),
               P2==P20+P22*lp2,
               diff(P2,r)==diff(P20+P22*lp2,r),
               diff(P2,r,r)==diff(P20+P22*lp2,r,r),
               diff(P2,th)==diff(P20+P22*lp2,th),
               diff(P2,th,th)==diff(P20+P22*lp2,th,th),
               diff(P2,th,r)==diff(P20+P22*lp2,th,r),
               diff(P2,r,th)==diff(P20+P22*lp2,r,th)]

for ii in range(0,len(sus_descomp)):
    ddG = ddG.subs(sus_descomp[ii])
    ddT = ddT.subs(sus_descomp[ii])

ddG = ddG.simplify_full()
ddT = ddT.simplify_full()

In [16]:
sus_legendre = [sin(th)^2 == (2/3)*(1-LP2),
                cos(th)^2 == (2*LP2+1)/3,
                cos(th) == LP1]

### <center> $\boxed{l=0}$ </center>

### <center> $G^{\prime\prime}_{00} = \kappa T^{\prime\prime}_{00}$ </center>

In [17]:
ddEq_00 = ddG[0,0] - kappa*ddT[0,0] == 0
E20_value_00 = solve(ddEq_00,E20)[0].rhs().simplify_full()

for ii in range(0,len(sus_legendre)):
    E20_value_00 = E20_value_00.subs(sus_legendre[ii])

E20_value_00 = E20_value_00.subs(LP1==0).subs(LP2==0).simplify_full()

ddG = ddG.subs(E20==E20_value_00)
ddT = ddT.subs(E20==E20_value_00)
sus_func.extend([E20==E20_value_00])
    
E20_paper = (4/r^2*diff(r*e^(-lamb)*m0,r)\
             +8/3*r*j*diff(j,r)*(omega-Omega1)^2\
             -1/3*j^2*r^2*diff(omega,r)^2)/(8*pi)

E20_check = E20_value_00 - E20_paper

show(LatexExpr("E^{(2)}_0 \\rightarrow"), bool(E20_check==0))

E^{(2)}_0 \rightarrow True

### <center> $G^{\prime\prime}_{11} = \kappa T^{\prime\prime}_{11}$ </center>

In [18]:
ddEq_11 = ddG[1,1] - kappa*ddT[1,1] == 0
P20_value = solve(ddEq_11,P20)[0].rhs().simplify_full()

for ii in range(0,len(sus_legendre)):
    P20_value = P20_value.subs(sus_legendre[ii])
    
P20_value = P20_value.subs(LP1==0).subs(LP2==0).simplify_full()
dP20_value = diff(P20_value,r)
for ii in range(0,len(sus_func)):
    dP20_value = dP20_value.subs(sus_func[ii])
    
ddG = ddG.subs(diff(P20,r)==dP20_value).subs(P20==P20_value)
ddT = ddT.subs(diff(P20,r)==dP20_value).subs(P20==P20_value)
sus_func.extend([diff(P20,r)==dP20_value,
                 P20==P20_value])

P20_paper = (4/r^2*(r*e^(-lamb)*diff(h0,r)-m0*(8*pi*r^2*P+1))\
             +1/3*r^2*j^2*diff(omega,r)^2)/(8*pi)

for ii in range(0,len(sus_func)):
    P20_paper = P20_paper.subs(sus_func[ii])

P20_check = P20_value - P20_paper

show(LatexExpr("P^{(2)}_0 \\rightarrow"), bool(P20_check==0))

P^{(2)}_0 \rightarrow True

### <center> $G^{\prime\prime}_{22} = \kappa T^{\prime\prime}_{22}$    y    $G^{\prime\prime}_{33} = \kappa T^{\prime\prime}_{33}$ </center>

In [19]:
ddEq_22 = ddG[2,2] - kappa*ddT[2,2] == 0
ddh0_value_22 = solve(ddEq_22,diff(h0,r,r))[0].rhs().simplify_full()

ddEq_33 = ddG[3,3] - kappa*ddT[3,3] == 0
ddh0_value_33 = solve(ddEq_33,diff(h0,r,r))[0].rhs().simplify_full()

for ii in range(0,len(sus_legendre)):
    ddh0_value_22 = ddh0_value_22.subs(sus_legendre[ii])
    ddh0_value_33 = ddh0_value_33.subs(sus_legendre[ii])

ddh0_value_22 = ddh0_value_22.subs(LP1==0).subs(LP2==0).simplify_full()
ddh0_value_33 = ddh0_value_33.subs(LP1==0).subs(LP2==0).simplify_full()

ddG = ddG.subs(diff(h0,r,r)==ddh0_value_22)
ddT = ddT.subs(diff(h0,r,r)==ddh0_value_22)
sus_func.extend([diff(h0,r,r)==ddh0_value_22])

m2h2 = ddh0_value_22 - ddh0_value_33 == 0
m2_value = solve(m2h2,m2)[0].rhs().simplify_full()

m2_paper = 1/6*r^4*j^2*diff(omega,r)^2\
          -1/3*r^3*diff(j^2,r)*(omega-Omega1)^2\
          -h2

dm2_value = diff(m2_value,r)
ddm2_value = diff(m2_value,r,r)
for ii in range(0,len(sus_func)):
    dm2_value = dm2_value.subs(sus_func[ii])
    ddm2_value = ddm2_value.subs(sus_func[ii])
    m2_paper = m2_paper.subs(sus_func[ii])

ddG = ddG.subs(diff(m2,r,r)==ddm2_value).subs(diff(m2,r)==dm2_value).subs(m2==m2_value)
ddT = ddT.subs(diff(m2,r,r)==ddm2_value).subs(diff(m2,r)==dm2_value).subs(m2==m2_value)
sus_func.extend([diff(m2,r,r)==ddm2_value,
                 diff(m2,r)==dm2_value,
                 m2==m2_value])

m2_check = m2_value - m2_paper

show(LatexExpr("m_2 \\rightarrow"), bool(m2_check==0))

m_2 \rightarrow True

### <center> Barotropic equation of state </center>

In [20]:
EOS = E2*diff(P,r) - P2*diff(E,r) == 0

for ii in range(0,len(sus_descomp)):
    EOS = EOS.subs(sus_descomp[ii])
EOS = EOS.simplify_full()

for ii in range(0,len(sus_legendre)):
    EOS = EOS.subs(sus_legendre[ii])

EOS = EOS.subs(LP1==0).subs(LP2==0)

PT0 = P20/(2*(E+P))

E20_value_EOS = solve(EOS, E20)[0].rhs()

dm0_value = E20_value_EOS - E20_value_00 == 0
dm0_value = solve(dm0_value, diff(m0,r))[0].rhs()

dm0_paper = - diff(r*e^(-lamb)*m0,r)\
            + 4*pi*r^2*diff(E,r)/diff(P,r)*(E+P)*PT0\
            + 1/12*j^2*r^4*diff(omega,r)^2\
            - 2/3*r^3*j*diff(j,r)*(omega-Omega1)^2  == 0

dm0_paper = solve(dm0_paper, diff(m0,r))[0].rhs()

ddm0_value = diff(dm0_value,r)
for ii in range(0,len(sus_func)):
    dm0_value = dm0_value.subs(sus_func[ii])
    ddm0_value = ddm0_value.subs(sus_func[ii])
    dm0_paper = dm0_paper.subs(sus_func[ii])
    
ddG = ddG.subs(diff(m0,r,r)==diff(dm0_value,r)).subs(diff(m0,r)==dm0_value)
ddT = ddT.subs(diff(m0,r,r)==diff(dm0_value,r)).subs(diff(m0,r)==dm0_value)
sus_func.extend([diff(m0,r,r)==diff(dm0_value,r),
                 diff(m0,r)==dm0_value])

dm0_check = dm0_value - dm0_paper
show(LatexExpr("m_{0}' \\rightarrow"), bool(dm0_check==0))

m_{0}' \rightarrow True

### <center> Ecuación 62 </center>

In [21]:
eq_62 = diff(PT0, r)\
        + 4*pi*(E+P)*r^2*PT0/(r-2*M)\
        + (r*e^(-lamb)*m0)*r^2/(r-2*M)^2*(8*pi*P+1/r^2)\
        - r^4*j^2/(12*(r-2*M))*diff(omega,r)^2\
        - 1/3*diff((r^3*j^2*(omega-Omega1)^2)/(r-2*M),r)

for ii in range(0,len(sus_func)):
    eq_62 = eq_62.subs(sus_func[ii])

show(LatexExpr("\\text{Ecuación (62) }\\rightarrow"), bool(eq_62==0))

\text{Ecuación (62) }\rightarrow True

### <center> Ecuación 64 </center>

In [22]:
eq_64 = PT0+h0-1/3*r^2*e^(-nu)*(omega-Omega1)^2
deq_64 = diff(eq_64, r)
for ii in range(0,len(sus_func)):
    deq_64 = deq_64.subs(sus_func[ii])

show(LatexExpr("\\text{Ecuación (64) }\\rightarrow"), bool(deq_64==0))

\text{Ecuación (64) }\rightarrow True

### <center> $\boxed{l=2}$ </center>

### <center> $G^{\prime\prime}_{22} = \kappa T^{\prime\prime}_{22}$    y    $G^{\prime\prime}_{33} = \kappa T^{\prime\prime}_{33}$ </center>

In [23]:
ddEq_22 = (ddG[2,2]-kappa*ddT[2,2] == 0).simplify_full()
ddEq_33 = ((ddG[3,3]-kappa*ddT[3,3] == 0)/sin(th)^2).simplify_full()

for ii in range(0,len(sus_legendre)):
    ddEq_22 = ddEq_22.subs(sus_legendre[ii])
    ddEq_33 = ddEq_33.subs(sus_legendre[ii])

ddEq_22 = (ddEq_22.subs(LP1==0) - ddEq_22.subs(LP1==0).subs(LP2==0))/LP2
P22_value_22 = solve(ddEq_22, P22)[0].rhs().simplify_full()

ddEq_33 = (ddEq_33.subs(LP1==0) - ddEq_33.subs(LP1==0).subs(LP2==0))/LP2
P22_value_33 = solve(ddEq_33, P22)[0].rhs().simplify_full()

for ii in range(0,len(sus_func)):
    P22_value_22 = P22_value_22.subs(sus_func[ii])
    P22_value_33 = P22_value_33.subs(sus_func[ii])
    
P22_value_22 = P22_value_22.simplify_full()
P22_value_33 = P22_value_33.simplify_full()

P22_check_22_33 = P22_value_22 - P22_value_33

show(LatexExpr("P^{(2)}_2\lvert_{\\theta\\theta}=P^{(2)}_2\lvert_{\\phi\\phi}\\rightarrow"), bool(P22_check_22_33==0))

P^{(2)}_2\lvert_{\theta\theta}=P^{(2)}_2\lvert_{\phi\phi}\rightarrow True

### <center> $G^{\prime\prime}_{00} = \kappa T^{\prime\prime}_{00}$ </center>

In [24]:
ddEq_00 = (ddG[0,0] - kappa*ddT[0,0] == 0).simplify_full()

for ii in range(0,len(sus_legendre)):
    ddEq_00 = ddEq_00.subs(sus_legendre[ii])

ddEq_00 = ddEq_00.simplify_full()
ddEq_00 = (ddEq_00.subs(LP1==0) - ddEq_00.subs(LP1==0).subs(LP2==0))/LP2

E22_value_00 = solve(ddEq_00,E22)[0].rhs()
for ii in range(0,len(sus_func)):
    E22_value_00 = E22_value_00.subs(sus_func[ii])
E22_value_00 = E22_value_00.simplify_full()

ddE22_value_00 = diff(E22_value_00,r,r)
dE22_value_00 = diff(E22_value_00,r)
for ii in range(0,len(sus_func)):
    ddE22_value_00 = ddE22_value_00.subs(sus_func[ii])
    dE22_value_00 = dE22_value_00.subs(sus_func[ii])

ddG = ddG.subs(diff(E22,r,r)==ddE22_value_00).subs(diff(E22,r)==dE22_value_00).subs(E22==E22_value_00)
ddT = ddT.subs(diff(E22,r,r)==ddE22_value_00).subs(diff(E22,r)==dE22_value_00).subs(E22==E22_value_00)
sus_func.extend([diff(E22,r,r)==ddE22_value_00,
                 diff(E22,r)==dE22_value_00,
                 E22==E22_value_00])

### <center> $G^{\prime\prime}_{11} = \kappa T^{\prime\prime}_{11}$ </center>

In [25]:
ddEq_11 = (ddG[1,1] - kappa*ddT[1,1] == 0).simplify_full()

for ii in range(0,len(sus_legendre)):
    ddEq_11 = ddEq_11.subs(sus_legendre[ii])

ddEq_11 = (ddEq_11.subs(LP1==0) - ddEq_11.subs(LP1==0).subs(LP2==0))/LP2

P22_value_11 = solve(ddEq_11,P22)[0].rhs()
for ii in range(0,len(sus_func)):
    P22_value_11 = P22_value_11.subs(sus_func[ii])
P22_value_11 = P22_value_11.simplify_full()

ddP22_value_11 = diff(P22_value_11,r,r)
dP22_value_11 = diff(P22_value_11,r)
for ii in range(0,len(sus_func)):
    ddP22_value_11 = ddP22_value_11.subs(sus_func[ii])
    dP22_value_11 = dP22_value_11.subs(sus_func[ii])

ddG = ddG.subs(diff(P22,r,r)==ddP22_value_11).subs(diff(P22,r)==dP22_value_11).subs(P22==P22_value_11)
ddT = ddT.subs(diff(P22,r,r)==ddP22_value_11).subs(diff(P22,r)==dP22_value_11).subs(P22==P22_value_11)
sus_func.extend([diff(P22,r,r)==ddP22_value_11,
                 diff(P22,r)==dP22_value_11,
                 P22==P22_value_11])

### <center> $G^{\prime\prime}_{12} = \kappa T^{\prime\prime}_{12}$ </center>

In [26]:
ddEq_12 = ddG[1,2]-kappa*ddT[1,2] == 0

dk2_value = solve(ddEq_12, diff(k2,r))[0].rhs().simplify_full()

dk2_paper = -h2*diff(nu,r)-diff(h2,r)\
            +(1/r+diff(nu,r)/2)*(-2/3*r^3*j*diff(j,r)*(omega-Omega1)^2\
                                 +1/6*j^2*r^4*diff(omega,r)^2)

for ii in range(0,len(sus_func)):
    dk2_value = dk2_value.subs(sus_func[ii])
    dk2_paper = dk2_paper.subs(sus_func[ii])

dk2_value = dk2_value.simplify_full()
ddk2_value = diff(dk2_value,r)
for ii in range(0,len(sus_func)):
    ddk2_value = ddk2_value.subs(sus_func[ii])

ddG = ddG.subs(diff(k2,r,r)==ddk2_value).subs(diff(k2,r)==dk2_value)
ddT = ddT.subs(diff(k2,r,r)==ddk2_value).subs(diff(k2,r)==dk2_value)
sus_func.extend([diff(k2,r,r)==ddk2_value,
                 diff(k2,r)==dk2_value])

dk2_check = (dk2_value - dk2_paper).simplify_full()

show(LatexExpr("k_2' \\rightarrow"), bool(dk2_check==0))

k_2' \rightarrow True

### <center> $G^{\prime\prime}_{22} = \kappa T^{\prime\prime}_{22}$ </center>

In [27]:
ddEq_22_00 = (ddG[2,2] - kappa*ddT[2,2] == 0).simplify_full()

for ii in range(0,len(sus_legendre)):
    ddEq_22_00 = ddEq_22_00.subs(sus_legendre[ii])

ddEq_22_00 = (ddEq_22_00.subs(LP1==0) - ddEq_22_00.subs(LP1==0).subs(LP2==0))/LP2
dh2_value = solve(ddEq_22_00, diff(h2,r))[0].rhs().simplify_full()

ddh2_value = diff(dh2_value,r)
for ii in range(0,len(sus_func)):
    ddh2_value = ddh2_value.subs(sus_func[ii])
ddh2_value = ddh2_value.simplify_full()

ddG = ddG.subs(diff(h2,r,r)==ddh2_value).subs(diff(h2,r)==dh2_value)
ddT = ddT.subs(diff(h2,r,r)==ddh2_value).subs(diff(h2,r)==dh2_value)
sus_func.extend([diff(h2,r,r)==ddh2_value,
                 diff(h2,r)==dh2_value])

dh2_paper = (-diff(nu,r)+r/((r-2*M)*diff(nu,r))*(8*pi*(E+P)-4*M/r^3))*h2\
            -4*(h2+k2)/(r*diff(nu,r)*(r-2*M))\
            +1/6*(1/2*r*diff(nu,r)-1/((r-2*M)*diff(nu,r)))*r^3*j^2*diff(omega,r)^2\
            -1/3*(1/2*r*diff(nu,r)+1/((r-2*M)*diff(nu,r)))*r^2*diff(j^2,r)*(omega-Omega1)^2

for ii in range(0,len(sus_func)):
    dh2_paper = dh2_paper.subs(sus_func[ii])

dh2_value = dh2_value.simplify_full()
dh2_check = dh2_value - dh2_paper

show(LatexExpr("h_{2}' \\rightarrow"), bool(dh2_check==0))

h_{2}' \rightarrow True

### <center> Ecuación 70 </center>

In [28]:
E22_check = E22_value_00 - 4/3*diff(E,r)/diff(nu,r)*(3*h2+e^(-nu)*r^2*(omega-Omega1)^2)
for ii in range(0,len(sus_func)):
    E22_check = E22_check.subs(sus_func[ii])
E22_check = E22_check.simplify_full()

show(LatexExpr("\\text{Ecuación (70) }\\rightarrow"), bool(E22_check==0))

\text{Ecuación (70) }\rightarrow True

### <center> Ecuación 71 </center>

In [29]:
P22_check = P22_value_11 + 2/3*(E+P)*(3*h2+e^(-nu)*r^2*(omega-Omega1)^2)
for ii in range(0,len(sus_func)):
    P22_check = P22_check.subs(sus_func[ii])
P22_check = P22_check.simplify_full()

show(LatexExpr("\\text{Ecuación (71) }\\rightarrow"), bool(P22_check==0))

\text{Ecuación (71) }\rightarrow True

### <center> Ecuación 73 </center>

In [30]:
PT2 = P22/(2*(E+P))

eq_73 = PT2+h2+1/3*e^(-nu)*r^2*(omega-Omega1)^2
for ii in range(0,len(sus_func)):
    eq_73 = eq_73.subs(sus_func[ii])

show(LatexExpr("\\text{Ecuación (73) }\\rightarrow"), bool(eq_73==0))

\text{Ecuación (73) }\rightarrow True

## <center> Matching </center>

In [93]:
var('a')

var('H0', latex_name="H_0")

var('s_k', latex_name="[k]")
var('s_h', latex_name="[h]")
var('s_m', latex_name="[m]")
var('s_dk', latex_name="[k']")
var('s_dh', latex_name="[h']")
var('s_dm', latex_name="[m']")

var('s_h0', latex_name="[h_{0}]")
var('s_m0', latex_name="[m_{0}]")
var('s_dh0', latex_name="[h_{0}']")
var('s_dm0', latex_name="[m_{0}']")

var('s_k2', latex_name="[k_{2}]")
var('s_h2', latex_name="[h_{2}]")
var('s_m2', latex_name="[m_{2}]")
var('s_dk2', latex_name="[k_{2}']")
var('s_dh2', latex_name="[h_{2}']")
var('s_dm2', latex_name="[m_{2}']")

var('s_dlamb', latex_name="[\\lambda']")
var('s_dnu', latex_name="[\\nu']")

var('dlamb_p', latex_name="\\lambda'_{+}(a)")
var('dlamb_m', latex_name="\\lambda'_{-}(a)")
var('dnu_p', latex_name="\\nu'_{+}(a)")
var('dnu_m', latex_name="\\nu'_{-}(a)")
var('ddnu_p', latex_name="\\nu''_{+}(a)")
var('ddnu_m', latex_name="\\nu''_{-}(a)")

var('s_ddnu', latex_name="[\\nu'']")

var('s_E', latex_name="[E]")
var('s_P', latex_name="[P]")

var('Q2', latex_name="\\hat{Q}_2")
var('Q20', latex_name="\\hat{Q}_{2(0)}")
var('Q22', latex_name="\\hat{Q}_{2(2)}")


var('dlamb_a', latex_name="\\lambda(a)'")
var('dnu_a', latex_name="\\nu(a)'")

nu_a = nu.subs(r==a)
lamb_a = -nu_a

In [94]:
eq_s_k = s_k == 0
eq_s_h = s_h == 1/2*H0 + 1/2*a*dnu_a*s_k
eq_s_m = s_m == a*s_dk + 1/4*e^(-lamb_a/2)*s_dlamb*Q2 + 1/2*(a*dlamb_m+2)*s_k
eq_s_dh = s_dh == 1/2*a*dnu_a*s_dk + 1/4*e^(-lamb_a/2)*s_ddnu*Q2 + 1/2*(a*ddnu_m+dnu_a)*s_k

In [95]:
sus_s_descomp = [s_m==s_m0+s_m2*LP2,
                 s_dm==s_dm0+s_dm2*LP2,
                 s_k==s_k2*LP2,
                 s_dk==s_dk2*LP2,
                 s_h==s_h0+s_h2*LP2,
                 s_dh==s_dh0+s_dh2*LP2,
                 Q2==Q20+Q22*LP2]

for ii in range(0,len(sus_s_descomp)):
    eq_s_k = eq_s_k.subs(sus_s_descomp[ii])
    eq_s_h = eq_s_h.subs(sus_s_descomp[ii])
    eq_s_m = eq_s_m.subs(sus_s_descomp[ii])
    eq_s_dh = eq_s_dh.subs(sus_s_descomp[ii])

In [111]:
M = a*(1-e^(-lamb_a))/2

#COMO ESTOS LOS HE ESCRITO YO A MANO, TENGO QUE ASEGURARME QUE LOS ÚLTIMOS TENGAN EN CUENTA LOS PRIMEROS
sus_s_func = [r == a,
              s_ddnu == (1+a*dnu_a/2)*e^(lamb_a)*8*pi*s_E,
              dnu_a == e^(-nu_a)*2*M/a^2,
              s_dnu == 0,
              s_dlamb == a*e^(lamb_a)*8*pi*s_E]

In [112]:
show((1+a*dnu_a/2)*e^(lamb_a)*8*pi*s_E)

4*pi*(a*dnu_a + 2)*s_E*e^(-nu(a))

In [113]:
s_h0_value = solve(eq_s_h, s_h0)[0].rhs()
s_h0_value = s_h0_value.subs(LP2==0)

s_h0_paper = H0/2
s_h0_check = s_h0_value - s_h0_paper

show(LatexExpr("[h_{0}]\\rightarrow"),bool(s_h0_check==0))

[h_{0}]\rightarrow True

In [114]:
s_m0_value = solve(eq_s_m, s_m0)[0].rhs()
s_m0_value = s_m0_value.subs(LP2==0)

s_m0_paper = 2*pi*s_E*e^(-nu_a/2)*a*Q20
s_m0_check = s_m0_value - s_m0_paper

for ii in range(0,len(sus_s_func)):
    s_m0_value = s_m0_value.subs(sus_s_func[ii])
    s_m0_check = s_m0_check.subs(sus_s_func[ii])
    
sus_s_func.extend([s_m0==s_m0_value])
    
show(LatexExpr("[m_{0}]\\rightarrow"),bool(s_m0_check==0))

[m_{0}]\rightarrow True

In [115]:
s_dh0_value = solve(eq_s_dh, s_dh0)[0].rhs()
s_dh0_value = s_dh0_value.subs(LP2==0)

s_dh0_paper = (a-M)/(a*(a-2*M))*s_m0
s_dh0_check = s_dh0_value - s_dh0_paper

for ii in range(0,len(sus_s_func)):
    s_dh0_check = s_dh0_check.subs(sus_s_func[ii])
    s_dh0_value = s_dh0_value.subs(sus_s_func[ii])
    s_dh0_paper = s_dh0_paper.subs(sus_s_func[ii])

show(s_dh0_value)
show(s_dh0_paper)

show(LatexExpr("[h'_{0}]\\rightarrow"),bool(s_dh0_check==0))

1/4*Q20*s_ddnu*e^(1/2*nu(a))

1/2*(a*(e^nu(a) - 1) + 2*a)*s_m0/((a*(e^nu(a) - 1) + a)*a)

-pi*((e^nu(a) - 1)*e^(-nu(a)) - 2)*Q20*s_E*e^(-1/2*nu(a))

pi*(a*(e^nu(a) - 1) + 2*a)*Q20*s_E*e^(-1/2*nu(a))/(a*(e^nu(a) - 1) + a)

[h'_{0}]\rightarrow True

In [116]:
show(s_dh0_value.subs(dnu_a == e^(-nu_a)*2*M/a^2).subs(r==a).simplify_full())

(pi*Q20*e^nu(a) + pi*Q20)*s_E*e^(-3/2*nu(a))

In [117]:
log(e)

1

In [103]:
show(sus_s_func[1])

dnu_a == -(e^nu(a) - 1)*e^(-nu(a))/a